In [1]:
from flask import Flask, jsonify, request
from flask_cors import CORS  # Import CORS
import tensorflow as tf
import numpy as np
import json
import joblib

In [2]:
model = tf.keras.models.load_model("route_prediction_ann_v2.h5")

In [3]:
# Load the saved scalers
scaler_X = joblib.load('scaler_X.pkl')
scaler_y = joblib.load('scaler_y.pkl')

In [4]:
with open("traffic_data.json") as f:
    traffic_data = json.load(f)

In [5]:
# Initialize Flask app
app = Flask(__name__)
CORS(app)

In [ ]:
# API endpoint to get predictions
@app.route('/api/getPrediction', methods=['GET'])
def get_prediction():
    # Get query parameters (starting location and destination)
    start_loc = request.args.get('start')
    dest_loc = request.args.get('dest')

    if not start_loc or not dest_loc:
        return jsonify({"error": "Please provide both 'start' and 'dest' parameters."}), 400

    # Check if the route exists in the traffic data
    try:
        route_info = traffic_data[start_loc][dest_loc]
    except KeyError:
        return jsonify({"error": "No traffic data available for this route."}), 404
    
    # Extract features from the JSON data
    distance = route_info["distance"]
    num_signals = route_info["num_signals"]
    vehicle_count = route_info["vehicle_count"]
    red_light_delay = route_info["red_light_delay"]
    current_pollution = route_info["current_pollution"]

    # Compute rush factor and pollution exposure
    rush_factor_car = vehicle_count / 300  
    rush_factor_bike = rush_factor_car * 0.8
    rush_factor_cycle = 0  
    rush_factor_bus = 0  
    rush_factor_metro = 0.4  
    pollution_exposure_car = current_pollution
    pollution_exposure_bike = pollution_exposure_car * 1.2
    pollution_exposure_cycle = pollution_exposure_car * 1.5
    pollution_exposure_bus = pollution_exposure_car * 0.8
    pollution_exposure_metro = pollution_exposure_car * 0.5

    # Prepare input data for the model
    input_data = np.array([[distance, num_signals, red_light_delay, vehicle_count,
                            rush_factor_car, rush_factor_bike, rush_factor_cycle, rush_factor_bus, rush_factor_metro,
                            pollution_exposure_car, pollution_exposure_bike, pollution_exposure_cycle,
                            pollution_exposure_bus, pollution_exposure_metro]])

    # Normalize the input data (using the scaler used during training)
    input_scaled = scaler_X.transform(input_data)

    # Make predictions with the model
    prediction_scaled = model.predict(input_scaled)

    # Reverse the normalization for the output
    output = scaler_y.inverse_transform(prediction_scaled)
    # Convert the output to regular float types (to avoid JSON serialization errors)
    output_floats = output[0].tolist()  # Convert numpy array to a list of floats


    # Prepare the result as a dictionary
    result = {
        "Time_Car": float(output_floats[0]),
        "Time_Bike": float(output_floats[1]),
        "Time_Cycle": float(output_floats[2]),
        "Time_Bus": float(output_floats[3]),
        "Time_Metro": float(output_floats[4]),
        "Expense_Car": float(output_floats[5]),
        "Expense_Bike": float(output_floats[6]),
        "Expense_Cycle": float(output_floats[7]),
        "Expense_Bus": float(output_floats[8]),
        "Expense_Metro": float(output_floats[9]),
        "Pollution_Car": float(output_floats[10]),
        "Pollution_Bike": float(output_floats[11]),
        "Pollution_Cycle": float(output_floats[12]),
        "Pollution_Bus": float(output_floats[13]),
        "Pollution_Metro": float(output_floats[14])
    }

    # Return the result as a JSON response
    return jsonify(result)

# Run the Flask app
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)  # use_reloader=False to avoid restarting the server during testing

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Feb/2025 21:32:44] "OPTIONS /api/getPrediction?start=Connaught%20Place&dest=Karol%20Bagh HTTP/1.1" 200 -
C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step


INFO:werkzeug:127.0.0.1 - - [03/Feb/2025 21:32:45] "GET /api/getPrediction?start=Connaught%20Place&dest=Karol%20Bagh HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Feb/2025 21:32:58] "OPTIONS /api/getPrediction?start=Connaught%20Place&dest=AIIMS HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [03/Feb/2025 21:32:58] "GET /api/getPrediction?start=Connaught%20Place&dest=AIIMS HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Feb/2025 21:33:07] "OPTIONS /api/getPrediction?start=Connaught%20Place&dest=Dwarka HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [03/Feb/2025 21:33:07] "GET /api/getPrediction?start=Connaught%20Place&dest=Dwarka HTTP/1.1" 200 -
